In [ ]:
#Stockage de trucs annexes :


# Filtrer un pays
path_data=path+'/GBIF_Philippines/'
    
carte_maille_philippines = gpd.read_file(path_data+'\\SIG\\country_grid_'+ecosysteme+'_'+str(grid_size_km)+'km.geojson')  # Remplace par le chemin vers ton fichier

df_biodiv_philippines=df_biodiv[df_biodiv[cle_geo].isin(carte_maille_philippines[cle_geo])]


In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt
# Charger les données
path = 'C:/Users/anormand/Documents/Projet Python/Biodiv/Data' 
world_terrestre = gpd.read_file(path + "/SIG_global/world-administrative-boundaries.geojson")

# Créer la figure et l'axe
fig, ax = plt.subplots(figsize=(12, 8))

# Tracer les données géographiques
world_terrestre.plot(ax=ax, edgecolor='black', facecolor='none', alpha=0.7)

# Ajouter des méridiens et des parallèles à intervalles réguliers
# Définir les intervalles
meridians = np.arange(-180, 181, 15)  # Tous les 30 degrés
parallels = np.arange(-90, 91, 15)   # Tous les 30 degrés

# Tracer les méridiens
for meridian in meridians:
    ax.plot([meridian, meridian], [-90, 90], color='gray', linestyle='--', linewidth=0.5)

# Tracer les parallèles
for parallel in parallels:
    ax.plot([-180, 180], [parallel, parallel], color='gray', linestyle='--', linewidth=0.5)

# Ajouter des étiquettes pour les méridiens et les parallèles
for meridian in meridians:
    ax.text(meridian, -95, f"{meridian}°", color='black', ha='center', fontsize=8)
for parallel in parallels:
    ax.text(-185, parallel, f"{parallel}°", color='black', va='center', fontsize=8)

# Ajouter un titre
ax.set_title('Carte du monde avec méridiens et parallèles', fontsize=14)

# Ajuster les limites
ax.set_xlim(-180, 180)
ax.set_ylim(-90, 90)

# Afficher la carte
plt.show()


In [ ]:
# Trop lente 
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np

def resolve_overlapping_geometries(df):
    """
    Resolves overlapping geometries by assigning intersections randomly to one of the geometries
    and removing it from the other.
    
    Parameters:
        df (GeoDataFrame): A GeoDataFrame with a 'geometry' column.
        
    Returns:
        GeoDataFrame: A GeoDataFrame with resolved geometries.
    """
    # Ensure input is a GeoDataFrame
    if not isinstance(df, gpd.GeoDataFrame):
        raise ValueError("Input data must be a GeoDataFrame.")

    # Reset index for easier row access
    df = df.reset_index(drop=True)

    total_pairs = len(df) * (len(df) - 1) // 2  # Total number of pairs to compare
    current_pair = 0  # To keep track of the current pair being processed
    last_percent = -1  # Variable to track the last displayed percent

    for i in range(len(df)):
        for j in range(i + 1, len(df)):
            geom1 = df.loc[i, "geometry"]
            geom2 = df.loc[j, "geometry"]
            
            # Check if geometries intersect
            if geom1.intersects(geom2):
                intersection = geom1.intersection(geom2)
                
                # Randomly decide who keeps the intersection
                if np.random.rand() > 0.5:
                    # Assign intersection to geom1 and remove it from geom2
                    df.loc[i, "geometry"] = geom1
                    df.loc[j, "geometry"] = geom2.difference(intersection)
                else:
                    # Assign intersection to geom2 and remove it from geom1
                    df.loc[j, "geometry"] = geom2
                    df.loc[i, "geometry"] = geom1.difference(intersection)
            
            current_pair += 1
            # Calculate percentage of progress
            percent = int((current_pair / total_pairs) * 100)
            
            # Display progress only if the percentage has changed
            if percent > last_percent:
                print(f"Progress: {percent}%")
                last_percent = percent
    
    # Remove empty geometries
    df = df[df.geometry.is_empty == False]
    
    return df

# Exemple d'utilisation
# df = gpd.read_file("chemin/vers/votre/fichier.geojson")
# df_resolved = resolve_overlapping_geometries(df)
carte_maille_test=resolve_overlapping_geometries(carte_maille)